### Crewai Hello World!

In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

import warnings
warnings.filterwarnings('ignore') 


from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from IPython.display import Markdown
from textwrap import dedent


watsonx_llm = LLM(
    model="watsonx/mistralai/mistral-large", # "watsonx/meta-llama/llama-3-3-70b-instruct",
    max_tokens=16384,
    temperature=0,
)


/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_too

In [2]:
import requests
from crewai_tools import ScrapeWebsiteTool#, FileWriterTool, TXTSearchTool

In [3]:
scrape_tool = ScrapeWebsiteTool(website_url="https://en.wikipedia.org/wiki/Artificial_intelligence")

# extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content

Artificial intelligence - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Naviga


In [4]:
# from fileinput import filename

# from sympy import content, true


# file_writer_tool = FileWriterTool()

# # write content to a file
# result = file_writer_tool.run(filename='ai.txt', content=text, directory='.', overwrite="True")

# result

In [5]:
from crewai_tools import TXTSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults

# search_tool = TXTSearchTool(txt='./ai.txt')
search_tool = TavilySearchResults(max_results=5)


In [6]:
from crewai import Agent, Task, Crew, Process

data_analyst = Agent(
    role="Educator",
    goal=f'Based on the context provided answer the question.',
    backstory="You are a data expert",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="understand the topic of Natural Language Processing and summarize it for me",
    agent=data_analyst,
    expected_output="I want the response to be as brief as possible."
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

ValidationError: 1 validation error for Agent
tools
  Value error, Invalid tool type: <class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>. Tool must be an instance of BaseTool or an object with 'name', 'func', and 'description' attributes. [type=value_error, input_value=[TavilySearchResults(api_...cretStr('**********')))], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [7]:
# ### Web Search tool
# os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY", None)
# os.environ['TOKENIZERS_PARALLELISM']='true'

# from langchain_community.tools.tavily_search import TavilySearchResults
# web_search_tool = TavilySearchResults(k=3)

from crewai_tools import SerperDevTool
from crewai import Agent, Task, Crew, Process


web_search_tool = SerperDevTool()

qa_agent = Agent(
    llm=watsonx_llm,
    role="Educator",
    goal=f'Based on the context provided answer the question.',
    backstory="You are a data expert",
    verbose=True,
    allow_delegation=False,
    tools=[web_search_tool]
)

nlp_task = Task(
    description="understand the topic of Natural Language Processing and summarize it for me",
    # agent=data_analyst, # commentic agent definition, let the manager decide
    expected_output="I want the response to be as brief as possible.",
    agent=qa_agent
)

calculator_agent = Agent(
    llm=watsonx_llm,
    role="Calculator",
    goal=f'you calculate things',
    backstory="you love math",
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="tell me what 123*435 is?",
    expected_output="Calculate this",
    agent=calculator_agent
)

crew = Crew(
    tasks=[nlp_task, math_task],
    agents=[qa_agent, calculator_agent],
    # process=Process.hierarchical,
    # manager_llm=llm,
    verbose=True
)

output = crew.kickoff()

# Agent: Educator
## Task: understand the topic of Natural Language Processing and summarize it for me


# Agent: Educator
## Thought: Thought: To understand and summarize the topic of Natural Language Processing (NLP), I need to gather comprehensive information about it.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Natural Language Processing\"}"
## Tool Output: 
{'searchParameters': {'q': 'Natural Language Processing', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Natural language processing - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Natural_language_processing', 'snippet': 'Major tasks in natural language processing are speech recognition, text classification, natural-language understanding, and natural-language generation.', 'position': 1}, {'title': 'What Is NLP (Natural Language Processing)? - IBM', 'link': 'https://www.ibm.com/think/topics/natural-language-processing', 'snippet': 'NLP is a subfield of c

### langchain + crewai

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun

search_ddg_tool = DuckDuckGoSearchRun()
search_ddg_tool.run("capital of Turkey?")

"Ankara, city, capital of Turkey, situated in the northwestern part of the country. It lies about 125 miles south of the Black Sea, near the confluence of the Hatip, Ince Su, and Cubek streams. Learn more about Ankara's history, economy, and attractions. Turkey is home to over 85 million people; [8] most are ethnic Turks, while ethnic Kurds are the largest ethnic minority. [5] Officially a secular state, Turkey has a Muslim-majority population. Ankara is Turkey's capital and second-largest city. Istanbul is its largest city and economic center. Other major cities include İzmir, Bursa, and Antalya. Turkey (Türkiye) is a country that occupies a unique geographic position, lying partly in Asia and partly in Europe and serving as both a bridge and a barrier between them. The modern Turkish republic was founded in 1923 after the collapse of the Ottoman Empire, and its capital is Istanbul (formerly Constantinople). Ankara, formerly known as Ancyra and Angora, is the capital of Turkey, locate

In [9]:
from crewai.tools import tool

@tool("DuckDuckGoSearch")
def ddg_search(search_query: str):
    """Search the web for information on the given query"""
    return search_ddg_tool.run(search_query)

In [10]:
researcher = Agent(
    role="Senior Research Analyst",
    goal="Ucover cutting-edge developments in AI and Data Science",
    backstory="""You work at a leading tech think tank. 
    Your experties lies in identifying emerging trends.
    You have a knack for dissecting compelx data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    tools=[ddg_search],
    llm=watsonx_llm
)

writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned content strategist, known for your insigntful and engagging articles.
    You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
    llm=watsonx_llm
)

In [11]:
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2025. 
    Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog post that highlights the most significant AI advancements.
    Your post should be informative yet accessible, catering to tech-savvy audiance.
    Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of atleat 4 paragraphs",
    agent=writer
)

In [12]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    process=Process.sequential,
)

2025-03-03 14:21:08,136 - 8697497664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [13]:
result = crew.kickoff()

print("---------------------")
print(result)

# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2025. 
    Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: To conduct a comprehensive analysis of the latest advancements in AI in 2025, I need to gather information on key trends, breakthrough technologies, and potential industry impacts. I will start by searching for the latest advancements in AI in 2025.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"latest advancements in AI in 2025\"}"
## Tool Output: 
Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to advanced AI legislation and sustainable AI initiatives. 3. It's boom time for AI in science One of the most exciting uses for AI is speeding up discovery in the natural sciences. Perhaps the greatest vindication of AI's potential on this front came ... Randy has just

In [14]:
from IPython.display import Markdown
Markdown(result.raw)

### Key Trends and Breakthrough Technologies in AI for 2025

#### Augmented Working
In 2025, AI will significantly enhance human capabilities, leading to more efficient and productive work environments. Companies like Microsoft and Google are integrating advanced AI tools into their productivity suites. For instance, Microsoft's "Copilot" feature in Office 365 uses AI to assist with writing emails, creating presentations, and even coding. This augmented working environment allows employees to focus on creative and strategic tasks while AI handles repetitive and mundane work.

#### Real-Time Decision-Making
AI systems will provide instantaneous insights and recommendations, enabling faster and more accurate decision-making. Retail giants like Amazon are using AI to make real-time pricing and inventory decisions. AI algorithms analyze customer behavior, market trends, and competitor actions to adjust prices and stock levels in real-time, maximizing profitability and customer satisfaction.

#### Advanced AI Legislation
There will be an increased focus on ethical AI development and regulatory frameworks to ensure responsible use of AI technologies. Governments worldwide are implementing stricter regulations to govern AI usage. The EU's AI Act, for instance, aims to classify AI systems based on risk and enforce transparency and accountability.

#### Sustainable AI Initiatives
Emphasis will be placed on developing AI solutions that are environmentally friendly and contribute to sustainability goals. Tech companies are investing in AI solutions to reduce their carbon footprint. Google's DeepMind AI is being used to optimize energy consumption in data centers, leading to significant energy savings.

#### AI in Science
AI will accelerate discoveries in natural sciences, leading to breakthroughs in various fields. AI is being used to predict protein structures, which is crucial for drug discovery. A pharmaceutical company used AI to analyze genetic data and identify potential drug targets for rare diseases, accelerating the drug discovery process by 50%.

#### Generative AI
Significant advancements in generative AI for creating content, including text, images, and videos, will revolutionize industries like entertainment, marketing, and design. Companies like Adobe are integrating generative AI into their creative tools to assist designers and artists. A fashion brand used generative AI to design a new clothing line, creating unique and stylish designs that were well-received by customers.

#### AI-Driven Security
There will be an increased use of AI in cybersecurity to combat AI-driven attacks, with a projected 300% rise in such attacks by 2025. AI is being used to enhance cybersecurity by detecting and responding to threats in real-time. Companies like Darktrace use AI to monitor network activity and identify anomalies that could indicate a security breach.

#### AI Agents in Enterprise Applications
Embedding AI agents in enterprise applications for real-time data access and action beyond traditional capabilities will become more common. AI agents are being integrated into enterprise applications to automate routine tasks. For instance, AI-powered chatbots are being used to handle customer inquiries and provide support.

#### Autonomous Systems
Advancements in self-driving cars and other autonomous systems will improve safety and efficiency. Companies like Tesla are at the forefront of developing autonomous vehicles that can navigate roads safely and efficiently. A mining company used autonomous drones to survey and map mining sites, optimizing mining operations and resulting in a 30% increase in efficiency.

#### Integration with Quantum Computing
Combining AI with quantum computing to solve complex problems more effectively will be a significant trend. Companies like IBM are developing quantum AI systems to tackle challenges in fields like chemistry and materials science. A research institution used quantum AI to simulate molecular interactions for drug discovery, performing calculations that were previously impossible.

### Breakthrough Technologies

#### Generative AI
Enhanced capabilities in creating realistic and diverse content will revolutionize industries like entertainment, marketing, and design. Generative AI is being used to create realistic images, videos, and text. Companies like Adobe are integrating generative AI into their creative tools to assist designers and artists.

#### AI-Native Security Stacks
Real-time threat hunting and blockchain-based disinformation security will combat advanced cyber threats. AI-native security stacks are being developed to provide comprehensive protection against cyber threats. These stacks use AI to continuously monitor and adapt to new threats.

#### Quantum AI
Leveraging quantum computing to solve problems that are currently intractable for classical AI systems will push the boundaries of what is possible. Companies like Google and IBM are investing in quantum AI research to solve complex problems more effectively.

#### Ethical AI Development
Tools and frameworks to ensure AI systems are fair, transparent, and accountable will be crucial. Companies are placing a greater emphasis on ethical AI development to ensure that AI systems are fair, transparent, and accountable. Initiatives like the AI Ethics Guidelines by the European Commission are guiding this effort.

### Potential Industry Impacts

#### Healthcare
Improved diagnostics, personalized treatment plans, and enhanced patient care through AI-driven insights will transform healthcare. AI is being used to analyze medical data and provide personalized treatment plans, leading to better patient outcomes.

#### Finance
Enhanced fraud detection, risk management, and personalized financial services will be possible with AI. Financial institutions are using AI to detect fraudulent transactions and manage risk more effectively.

#### Manufacturing
Increased automation, predictive maintenance, and optimized supply chain management will revolutionize manufacturing. AI is being used to optimize production processes and reduce downtime, leading to increased efficiency.

#### Retail
Personalized shopping experiences, inventory management, and customer service improvements will be driven by AI. Retailers are using AI to analyze customer data and provide personalized shopping experiences.

#### Transportation
Safer and more efficient transportation systems through autonomous vehicles and smart traffic management will be a reality. AI is being used to optimize traffic flow and reduce congestion, leading to safer and more efficient transportation systems.

#### Entertainment
Revolutionized content creation and personalized user experiences will be possible with AI. Entertainment companies are using AI to create personalized content and enhance user experiences.

#### Cybersecurity
Enhanced protection against sophisticated cyber threats and real-time threat detection will be crucial. AI is being used to detect and respond to cyber threats in real-time, providing enhanced protection against sophisticated attacks.

These advancements and trends in AI for 2025 are set to reshape various industries, leading to more efficient, innovative, and sustainable solutions. As we move towards 2025, these trends and breakthrough technologies will continue to shape the future of AI and its impact on society.
```

### Trip Advisor Agent

In [15]:
from crewai import Agent

class TripAgents():

    def local_expert(self):
        return Agent(
            role="Local expert at this city",
            goal="Provide 2-3 insights about the selected city",
            backstory="""A knowledgeable local guide with extensive information about the city, it's attractions and customs""",
            tools=[web_search_tool, scrape_tool],
            verbose=True,
            llm=watsonx_llm
        )
    
    def travel_concierge(self):
        return Agent(
            role='Amazing Travel Concierge',
            goal="""Create the most amazing travel itineraries with budget and packing suggestions for the city""",
            backstory="""Specialist in travel planning and logistics with decades of experience""",
            tools=[],
            verbose=True,
            llm=watsonx_llm
        ) 

In [16]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(
        description= dedent(f"""
            I want you to create a comprehensive city guide.
            This guide should provide a thorough overview of what
            the city has to offer, including hidden gems, cultural
            hotspots, and must-visit landmarks.

            Trip Date: {range}
            Traveling from: {origin}
            Traveling to: {city}
            Traveler Interests: {interests}
            """),

        expected_output=dedent(f"""
            The final answer must be a comprehensive city guide,
            rich in cultural insights and practical tips,
            tailored to enhance the travel experience.

            """),
        agent=agent
    )

  def plan_task(self, agent, origin, city, interests, range):
    return Task(
        description=dedent(f"""
            Expand this guide into a a full 1-day travel
            itinerary with detailed per-day plans, including
            weather forecasts, places to eat, packing suggestions,
            and a budget breakdown.

            You MUST suggest actual places to visit, actual hotels
            to stay and actual restaurants to go to.

            Trip Date: {range}
            Traveling from: {origin}
            Traveling to: {city}
            Traveler Interests: {interests}
        """),
        expected_output=dedent(f"""
            Your final answer MUST be a complete expanded travel plan,
            formatted as markdown, encompassing a daily schedule,
            anticipated weather conditions, recommended clothing and
            items to pack, and a detailed budget, ensuring THE BEST
            TRIP EVER, Be specific and give it a reason why you picked
            # up each place, what make them special!
            """),
        agent=agent
    )

In [17]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

In [18]:
trip_crew = TripCrew('Bangalore', 'Paris', 'June 2025', 'art')
result = trip_crew.run()

2025-03-03 14:24:51,436 - 8697497664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Local expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: June 2025
Traveling from: Bangalore
Traveling to: Paris
Traveler Interests: art



# Agent: Local expert at this city
## Thought: To create a comprehensive city guide for Paris tailored to art enthusiasts, I need to gather information about the city's art scene, including famous museums, hidden gems, and cultural hotspots.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Paris art scene and cultural hotspots\"}"
## Tool Output: 
{'searchParameters': {'q': 'Paris art scene and cultural hotspots', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Paris Art Scene: Museums And Galleries Too Good To Miss - France', 'link': 'https://barefootblogger.com/paris-art-scene-museums-and-galleri

In [19]:
print(result)

# Comprehensive 1-Day Travel Itinerary for Paris: June 2025

## Introduction
Welcome to Paris, the City of Lights and a haven for art enthusiasts! This 1-day itinerary will provide you with a detailed plan to explore the rich art scene of Paris, including must-visit landmarks, hidden gems, and cultural hotspots.

## Weather Forecast for June 2025
June in Paris is typically pleasant with mild temperatures. Expect average highs of around 23°C (73°F) and lows of around 14°C (57°F). There may be occasional rain showers, so it's wise to pack a light rain jacket.

## Packing Suggestions
- **Clothing:** Light layers, including t-shirts, blouses, and a light sweater or jacket for cooler evenings.
- **Footwear:** Comfortable walking shoes.
- **Accessories:** A light rain jacket, sunglasses, and a hat.
- **Essentials:** Travel adapter, portable charger, reusable water bottle, and a small travel umbrella.
- **Art Supplies:** Sketchbook and pencils if you enjoy drawing.

## Budget Breakdown
- **Fl

In [20]:
Markdown(result.raw)

# Comprehensive 1-Day Travel Itinerary for Paris: June 2025

## Introduction
Welcome to Paris, the City of Lights and a haven for art enthusiasts! This 1-day itinerary will provide you with a detailed plan to explore the rich art scene of Paris, including must-visit landmarks, hidden gems, and cultural hotspots.

## Weather Forecast for June 2025
June in Paris is typically pleasant with mild temperatures. Expect average highs of around 23°C (73°F) and lows of around 14°C (57°F). There may be occasional rain showers, so it's wise to pack a light rain jacket.

## Packing Suggestions
- **Clothing:** Light layers, including t-shirts, blouses, and a light sweater or jacket for cooler evenings.
- **Footwear:** Comfortable walking shoes.
- **Accessories:** A light rain jacket, sunglasses, and a hat.
- **Essentials:** Travel adapter, portable charger, reusable water bottle, and a small travel umbrella.
- **Art Supplies:** Sketchbook and pencils if you enjoy drawing.

## Budget Breakdown
- **Flights:** Approximately €800 round trip from Bangalore to Paris.
- **Accommodation:** €200 for a night at a mid-range hotel.
- **Transportation:** €20 for a Paris Visite pass.
- **Food:** €50 for meals and snacks.
- **Entrance Fees:** €30 for museums and attractions.
- **Miscellaneous:** €50 for souvenirs and unexpected expenses.

**Total Estimated Budget:** €1150

## 1-Day Itinerary

### Morning

#### 8:00 AM - Breakfast at Café de Flore
Start your day with a traditional French breakfast at the iconic Café de Flore in Saint-Germain-des-Prés. This historic café was a favorite among artists and writers like Sartre and Hemingway.
- **Why:** Experience the intellectual and artistic atmosphere of Paris.

#### 9:30 AM - The Louvre Museum
Head to the Louvre Museum, the world's largest and most visited art museum. Don't miss the Mona Lisa, the Winged Victory, and the Venus de Milo.
- **Why:** A must-visit for any art lover, housing thousands of works of art.

### Afternoon

#### 12:30 PM - Lunch at Le Comptoir du Relais
Enjoy a delicious French lunch at Le Comptoir du Relais, known for its traditional bistro dishes.
- **Why:** Authentic French cuisine in a charming setting.

#### 2:00 PM - Musée d'Orsay
Visit the Musée d'Orsay, located in a former railway station. Explore the vast collection of Impressionist and Post-Impressionist masterpieces, including works by Monet, Degas, and Van Gogh.
- **Why:** A stunning collection of Impressionist art in a unique setting.

### Evening

#### 5:00 PM - Centre Pompidou
Head to the Centre Pompidou, an iconic building housing the National Museum of Modern Art. Enjoy the stunning view of the city from its rooftop.
- **Why:** A cultural hub with a vast public library, cinemas, and performance spaces.

#### 7:00 PM - Dinner at Le Train Bleu
Have dinner at Le Train Bleu, a historic restaurant located in the Gare de Lyon train station. The opulent setting and delicious French cuisine make it a memorable experience.
- **Why:** A unique dining experience in a historic setting.

### Night

#### 9:00 PM - Eiffel Tower
End your day with a visit to the Eiffel Tower. Enjoy the breathtaking view of the city from the top and watch the tower sparkle every hour on the hour.
- **Why:** An iconic symbol of Paris and a must-visit for any traveler.

## Accommodation

### Hotel des Arts - Montmartre
Located in the charming neighborhood of Montmartre, this hotel offers a mix of historic charm and modern amenities.
- **Why:** Immersive experience in a vibrant and artistic neighborhood.

## Practical Tips

### Getting Around
Paris has an excellent public transportation system, including the Métro, buses, and trams. Consider getting a Paris Visite pass for unlimited travel within the city.

### Safety
Paris is generally safe, but like any big city, it's important to be cautious. Keep an eye on your belongings, especially in crowded tourist areas, and avoid poorly lit streets at night.

## Conclusion
Paris is a city that never fails to inspire and captivate. With its rich art scene, hidden gems, and cultural hotspots, it offers a unique blend of history, creativity, and charm. This 1-day itinerary will ensure you make the most of your time in the City of Lights. Enjoy your journey and let the City of Lights illuminate your artistic soul!

Bon voyage!

In [21]:
print("## Welcome to Trip Planner Crew")
print('-——————————')
location = input(
dedent("""
  From where will you be traveling from?
"""))
city = input(
dedent("""
  Where do you want to go?
"""))
date_range = input(
dedent("""
  What is the date range you are interested in traveling?
"""))
interests = input(
dedent("""
  What are some of your high level interests and hobbies?
"""))

trip_crew = TripCrew(location, city, date_range, interests)
result = trip_crew.run()

## Welcome to Trip Planner Crew
-——————————


2025-03-03 14:26:44,542 - 8697497664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Local expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: Sept 2025
Traveling from: Bangalore
Traveling to: SF
Traveler Interests: Tech, Art, City Landscape



# Agent: Local expert at this city
## Thought: To create a comprehensive city guide for San Francisco tailored to the traveler's interests in tech, art, and city landscapes, I need to gather information about the city's tech scene, art attractions, and notable landmarks.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"San Francisco tech attractions\"}"
## Tool Output: 
{'searchParameters': {'q': 'San Francisco tech attractions', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'A day in SF area. Interested in tech. Any recommendations? : r/AskSF', 'link': 'https://www.reddit.com/r

2025-03-03 14:27:45,570 - 8697497664 - serper_dev_tool.py-serper_dev_tool:170 - ERROR: Error making request to Serper API: HTTPSConnectionPool(host='google.serper.dev', port=443): Read timed out. (read timeout=10)




# Agent: Local expert at this city
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"San Francisco art attractions\"}"
## Tool Output: 
{'searchParameters': {'q': 'San Francisco art attractions', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Fine Arts Museums of San Francisco | de Young and Legion of Honor', 'link': 'https://www.famsf.org/', 'snippet': 'Explore our collections · American art. blue lake and mountain · Contemporary art. Sculpture of a woman with a beehive for a head in a garden · Photography.', 'position': 1, 'sitelinks': [{'title': 'De Young', 'link': 'https://www.famsf.org/visit/de-young'}, {'title': "What's on", 'link': 'https://www.famsf.org/whats-on'}, {'title': 'About', 'link': 'https://www.famsf.org/about'}, {'title': 'Women in art', 'link': 'https://www.famsf.org/stories/all?topic=52'}]}, {'title': 'San Francisco Arts & Culture', 'link': 'https://www.sftravel.com/things-to-do/arts-culture', 'snippet'

2025-03-03 14:28:00,967 - 8697497664 - llm.py-llm:348 - ERROR: LiteLLM call failed: litellm.InternalServerError: watsonxException - {"errors":[{"code":"downstream_request_failed","message":"Downstream vllm request failed: Internal Server Error","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"c0dd74325cab549a1d7241c1d4020f28","status_code":500}




LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.InternalServerError: watsonxException - {"errors":[{"code":"downstream_request_failed","message":"Downstream vllm request failed: Internal Server Error","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"c0dd74325cab549a1d7241c1d4020f28","status_code":500}


InternalServerError: litellm.InternalServerError: watsonxException - {"errors":[{"code":"downstream_request_failed","message":"Downstream vllm request failed: Internal Server Error","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"c0dd74325cab549a1d7241c1d4020f28","status_code":500}

In [ ]:
print("\n\n########################")
print("## Here is your Trip Plan")
print("########################\n")
Markdown(result.raw)



########################
## Here is your Trip Plan
########################



# Comprehensive 1-Day Travel Itinerary for San Francisco

## Introduction
Welcome to San Francisco! This 1-day itinerary is designed to give you a taste of the city's tech scene, art culture, and stunning landscapes.

## Weather Forecast for September 2025
- **Temperature:** High of 70°F (21°C), Low of 55°F (13°C)
- **Conditions:** Partly cloudy with morning fog, typical for San Francisco in September.

## Packing Suggestions
- **Clothing:** Dress in layers (t-shirt, sweater, light jacket) to adapt to changing temperatures.
- **Footwear:** Comfortable walking shoes.
- **Accessories:** Sunglasses, hat, and a light scarf.
- **Electronics:** Smartphone, portable charger, and a camera.
- **Miscellaneous:** Reusable water bottle, snacks, and a small backpack.

## Budget Breakdown
- **Accommodation:** $200 (Hotel Zephyr)
- **Transportation:** $20 (Clipper Card for public transit)
- **Food:** $100 ($30 breakfast, $40 lunch, $30 dinner)
- **Attractions:** $50 (museum entries and other attractions)
- **Miscellaneous:** $30 (souvenirs, snacks)
- **Total:** $400

## Detailed 1-Day Itinerary

### Morning

#### 8:00 AM - Breakfast at Brenda's French Soul Food
- **Location:** 652 Polk St, San Francisco, CA 94102
- **Why:** Known for its delicious Creole and French-inspired dishes, Brenda's is a great way to start your day.

#### 9:30 AM - Exploratorium
- **Location:** Pier 15, The Embarcadero, San Francisco, CA 94111
- **Why:** An interactive science museum that combines art, science, and human perception. It's a great place to explore the intersection of technology and creativity.

### Afternoon

#### 12:30 PM - Lunch at Boudin Bakery
- **Location:** 160 Jefferson St, San Francisco, CA 94133
- **Why:** Famous for its clam chowder in a sourdough bread bowl, Boudin Bakery is a San Francisco classic.

#### 1:30 PM - San Francisco Museum of Modern Art (SFMOMA)
- **Location:** 151 3rd St, San Francisco, CA 94103
- **Why:** One of the largest museums of modern and contemporary art in the United States, featuring works by renowned artists.

#### 3:30 PM - Mission District Murals
- **Location:** Various locations in the Mission District
- **Why:** Explore the vibrant street art and murals that adorn the walls of the Mission District. These murals reflect the neighborhood's rich cultural heritage and social issues.

### Evening

#### 5:30 PM - Golden Gate Bridge
- **Location:** US-101, San Francisco, CA 94129
- **Why:** An iconic symbol of San Francisco, the Golden Gate Bridge offers breathtaking views of the city and the bay.

#### 7:00 PM - Dinner at Gary Danko
- **Location:** 800 North Point St, San Francisco, CA 94109
- **Why:** A renowned French restaurant offering a unique dining experience with exquisite cuisine.

### Night

#### 9:00 PM - Twin Peaks
- **Location:** 501 Twin Peaks Blvd, San Francisco, CA 94114
- **Why:** Offering panoramic views of the city, Twin Peaks is a great spot for hiking and enjoying the natural beauty of San Francisco.

## Accommodation

### Hotel Zephyr
- **Location:** 250 Beach St, San Francisco, CA 94133
- **Why:** A modern and stylish hotel located near Fisherman's Wharf, offering easy access to many of the city's attractions.

## Practical Tips
- **Transportation:** Use the Clipper Card for easy access to public transit.
- **Weather:** Dress in layers to adapt to changing temperatures.
- **Safety:** Be cautious in crowded tourist areas and keep an eye on your belongings to avoid pickpocketing.

## Conclusion
This 1-day itinerary offers a taste of San Francisco's tech scene, art culture, and stunning landscapes. Enjoy your trip!